In [2]:
from langchain.schema import Document

In [3]:
type(Document)

pydantic._internal._model_construction.ModelMetaclass

In [5]:
Document

langchain_core.documents.base.Document

## Types of Loader

###### PDFLoader
###### CSVLoader
###### WebBaseLoader
###### DirectoryLoader

In [8]:
from langchain_core.documents import Document # same as above import

In [18]:
doc = Document(
    page_content = "This is a txt file data", 
    metadata={
        "source":"example.txt",
        "pages": "5",
        "author":"Gaurav"
    }
)

In [19]:
doc

Document(metadata={'source': 'example.txt', 'pages': '5', 'author': 'Gaurav'}, page_content='This is a txt file data')